# Tarea 2. Parámetros cosmológicos a partir del Fondo Cósmico de Microondas

El objetivo de esta tarea es analizar un mapa ficticio del CMB e identificar los parámetros cosmológicos usados para generarlo. Para esto deberán utilizar `CAMB` (https://camb.info/), en cualquiera de sus versiones.

## Parte 1. Generar un mapa del CMB

Para esta tarea deberán usar el espectro angular de potencias de temperatura provisto junto con este enunciado, disponible en el repositorio del curso, que se muestra a continuación:

<img src="./D_ell.png">

Este espectro corresponde a la componente `unlensed_scalar` entregada por `CAMB`.

Noten las unidades del eje vertical. Noten también que la curva empieza en $\ell=2$. ¿Qué pasa con $\ell=0,1$?

**La primera parte consiste en generar un mapa de $10^\circ\times10^\circ$ a partir de este espectro de potencias.** Este tamaño de mapa nos permite trabajar en la aproximación de cielo plano (de otro modo tendríamos que trabajar con esféricos armónicos). 

El espectro de potencias está en función de $\ell$, que se relaciona al ángulo en el cielo (y al número de onda, $k$) a través de $\ell=2\pi k=2\pi/\theta$, donde $\theta$ está en radianes. Podemos generar un mapa en espacio de Fourier,
$$
\mathbf{\tilde M}(\ell_x,\ell_y) = C\left(\sqrt{\ell_x^2+\ell_y^2}\right)
$$

Tengan en cuenta que este espectro de potencias es generado a partir de fluctuaciones primordiales Gaussianas, de manera que el mapa observado del CMB está dado por

$$
\mathbf{M}(\theta_x,\theta_y) =
    \int d\ell_x \int d\ell_y \exp\left[-2i(\vec\ell\cdot\vec\theta)\right] \, \mathbf{\tilde M}(\ell_x,\ell_y)\,\mathbf{\tilde G}(\ell_x,\ell_y)
$$

donde $\mathbf{\tilde G}(\ell_x,\ell_y)$ es la transformada de Fourier (en 2 dimensiones) de una campo Gaussiano aleatorio,

$$
\mathbf{\tilde G}(\ell_x,\ell_y) =
    \int d\ell_x \int d\ell_y \exp\left[-2i(\vec\ell\cdot\vec\theta)\right] \, \mathcal{N}(\mu,\sigma)
$$

(Para el valor de $\sigma$, consideren que éste determina la amplitud de las fluctuaciones *respecto del espectro de potencias $C_\ell$*.) Deberían obtener una imagen similar (pero no idéntica) la que se muestra a continuación:

<img src='./cmb_Tmap.png'>

Este mapa tiene píxeles de $0.5'$, correspondientes al tamaño de los píxeles en los mapas de ACT y SPT. Deberán comparar mapas de diferentes resoluciones usando distintos valores de cortes en $\ell$. Deberán además discutir cómo se compara este mapa con un mapa real del CMB: ¿qué elementos faltó considerar para tener un mapa realista a estas escalas y con esta resolución?

Noten que distintas realizaciones del campo Gaussiano de fluctuaciones primordiales entregarán distintos mapas con espectro de potencias idéntico. Por supuesto, no es necesario que implementen la transformada de Fourier ustedes mismos; pueden usar la Transformada de Fourier Rápida (FFT) para hacer los cálculos de arriba.

## Parte 2. Calcular el espectro de potencias

**En la segunda parte de esta tarea, tendrán que calcular el espectro de potencias a partir del mapa generado.**

### 2.1 Apodización

Si tomáramos la transformada de Fourier del mapa directamente, tendríamos problemas por la discontinuidad en los bordes del mapa. Para evitar esto, el mapa se **apodiza** - es decir, se multiplica por una función que lo haga decaer suavemente a cero en los bordes:

$$
\mathbf{M}_\mathrm{apod}(\theta_x,\theta_y) = \mathbf{M}(\theta_x,\theta_y) \circ \mathbf{W}(\theta_x,\theta_y)
$$

donde $\circ$ es el operador multiplicador por elementos (producto de Hadamard). La función a usar queda a su elección (pueden buscar alguna referencia o explorar distintas funciones); deberán justificar su elección y mostrar el mapa apodizado.

### 2.2 Espectro de potencias

El espectro de potencias se calcula tomando la transformada de Fourier (en 2d) del mapa apodizado, recordando que $\ell = 2\pi k \equiv 2\pi\sqrt{k_x^2+k_y^2}$, 
$$
\mathbf{\tilde M}(\ell_x,\ell_y) = \mathrm{FFT}\left(\mathbf{M}_\mathrm{apod}(\theta_x,\theta_y)\right)
$$

El espectro de potencias se calcula entonces promediando el mapa en anillos de $\ell$ constante (o dentro de un rango pequeño), y elevando esto al cuadrado (en el caso del auto-espectro).
$$
C_\ell\left(\sqrt{\ell_x^2+\ell_y^2}\right) = \langle\mathbf{\tilde M}(\ell_x,\ell_y)\rangle
$$

#### 2.2.1 Corrección de sesgos en el espectro de potencias

Al calcular el espectro de potencias como está arriba, estamos cometiendo ciertos errores debidos principalmente a la apodización y la resolución finita de nuestro mapa. Esto puede expresarse de la siguiente manera:
$$
\hat D_\ell = T_\ell*D_\ell + N_\ell
$$

donde $D_\ell$ es el verdadero espectro de potencias, $\hat D_\ell$ es el espectro de potencias estimado, $T_\ell$ es la *función de transferencia* y $N_\ell$ es el *sesgo de ruido*. Así, el espectro de potencias real corresponde a
$$
D_\ell = \frac{(\hat D_\ell - N_\ell)}{T_\ell}
$$

Estos sesgos pueden estimarse usando mapas simulados donde el espectro de potencias es conocido - justamente la situación aquí. Noten que en la primera parte hay una componente aleatoria - el campo primordial de fluctuaciones - que les permite generar mapas distintos a partir de un único espectro de potencias.

**Sesgo de ruido**

Noten que el sesgo de ruido existe aunque la señal verdadera sea cero - y así es exactamente como se estima: generando mapas que sólo contienen ruido (i.e., $\mathbf{\tilde G}(\ell_x,\ell_y)$); el espectro de potencias resultante es $N_\ell$.

**Función de transferencia**

La función de transferencia es una corrección multiplicativa al espectro de potencias, y depende principalmente del tamaño finito del mapa y la resolución del telescopio. la función de transferencia puede calcularse usando mapas simulados en los que se conoce el espectro de potencias que lo genera, y comparando el espectro resultante con el original. A pesar de ser generados a partir del mismo espectro de potencias, los mapas no serán los mismos por la componente aleatoria introducida arriba.

Con estos ingredientes ya pueden calcular el espectro de potencias. ¿Cómo se compara con el espectro entregado?

# Parte 3. Estimación de parámetros cosmológicos

**En la última parte deberán identificar los parámetros cosmológicos** aproximados con los que se generó el espectro de potencias inicial. Para esto, deberán usar `CAMB`, disponible como ejecutable escrito en `FORTRAN` y como paquete en `python`. Deberán generar espectros de potencias modificando los parámetros $\Omega_\mathrm{c}h^2$ y $\Omega_\mathrm{b}h^2$ (el resto de parámetros corresponden todos a los valores incorporados en el ejemplo [aquí](https://camb.readthedocs.io/en/latest/CAMBdemo.html)):

|      $A_s$       |  $n_s$  | $r$ | $H_0$ $(\mathrm{km\,s^{-1}\,Mpc^{-1}})$ |  $\sum m_\nu\,(\mathrm{eV})$ | $\Omega_k$ | $\tau$ |
|:----------------:|:-------:|:---:|:--------------------------------:|:-------------:|:----------:|:------:|
| $2\times10^{-9}$ | $0.965$ |  $0.00$  |             $67.5$               |     $0.06$    |   $0.00$   | $0.06$ | 

Deberán producir un "mapa" (opcionalmente de contornos) cuantificando la diferencia entre cada uno de los espectros resultantes y el espectro entregado en esta tarea, con alguna métrica de su elección, dependiendo de los valores de los dos parámetros mencionados. ¿Cuáles son los valores más probables para estos parámetros? Comenten sobre su resultado. Por ejemplo, ¿qué características identifican en el espectro de potencias que avalen los valores que encontraron? ¿Hay alguna degeneración entre estos parámetros, y a qué se debe?

# Referencias

Éstos son sólo algunos ejemplos:

* https://ui.adsabs.harvard.edu/abs/2017JCAP...06..031L/abstract
* https://ui.adsabs.harvard.edu/abs/2018ApJ...852...97H/references